# Minimize beam size and measure emittance on DIAG0

In [ ]:
# set up env for running on SLAC production servers
import os
os.environ['OMP_NUM_THREADS']=str(6)

In [ ]:
run_dir = '/home/physics/ml_tuning/20241111_DIAG0'

# Read pv info from YAML files

In [ ]:
import sys
import yaml
# sys.path.append("../../")
sys.path.append("../")

from common import get_pv_objects, save_reference_point, set_magnet_strengths, \
    measure_pvs

In [ ]:
pv_bounds = yaml.safe_load(open("pv_bounds.yml"))
pv_objects = get_pv_objects("tracked_pvs.yml")

In [ ]:
pv_bounds

# load reference point
Also define a function to write the reference values to the pvs

In [ ]:
reference = yaml.safe_load(open("reference.yml"))

def reset_pvs():
    set_magnet_strengths(reference, pv_objects, validate=False)

In [ ]:
reference

In [ ]:
reset_pvs()

In [ ]:
from lcls_tools.common.devices.reader import create_screen, create_magnet
from lcls_tools.common.measurements.screen_profile import ScreenBeamProfileMeasurement

screen = create_screen(area='DIAG0', name='OTRDG02')
device_measurement = ScreenBeamProfileMeasurement(device=screen)

# Test screen measurement

In [ ]:
from matplotlib import pyplot as plt
results = device_measurement.measure()
plt.imshow(results["raw_image"])
plt.imshow(results["processed_image"])

# Imports

In [ ]:
# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")
import torch
from xopt import Xopt
from xopt.vocs import VOCS
from xopt.evaluator import Evaluator
from xopt.numerical_optimizer import LBFGSOptimizer
from xopt.generators import UpperConfidenceBoundGenerator
from xopt.generators.bayesian.models.standard import StandardModelConstructor
import numpy as np
import random

# Evaluator

In [ ]:
import time
# define function to measure the total size on OTR4
def eval_beamsize(inputs):
    
    # set pvs and wait for BACT to settle to correct values (validate=True)
    set_magnet_strengths(inputs, pv_objects, validate=True)
    time.sleep(0.5)
    # measure all pvs - except for names in inputs
    results = measure_pvs(
        [name for name in pv_objects.keys() if name not in inputs], pv_objects 
    )

    # do some calculations
    results["time"] = time.time()

    # add beam size measurement to results dict
    beamsize_results = device_measurement.measure()
    results["Sx_mm"] = np.array(np.mean(beamsize_results["Sx"])) * 1e-3
    results["Sy_mm"] = np.array(np.mean(beamsize_results["Sy"])) * 1e-3

    #add beam size squared (mm^2) and total size (mm^2)
    results["xrms_sq"] = results["Sx_mm"]**2
    results["yrms_sq"] = results["Sy_mm"]**2
    results["total_size"] = np.sqrt(results["xrms_sq"] * results["yrms_sq"])
    results = beamsize_results | results
    return results

evaluator = Evaluator(function=eval_beamsize)

In [ ]:
eval_beamsize({})

# Vocs

In [ ]:
IMAGE_CONSTRAINTS = {
            "bb_penalty": ["LESS_THAN", 0.0],
        }

vocs = VOCS(
    variables = { # STOP!! NEED TO GET ACCURATE RANGES!!!!! THESE ARE PLACEHOLDER VALUES!!
        # 'QUAD:DIAG0:230:BCTRL': [-0.005, 0.005],
        # 'QUAD:DIAG0:270:BCTRL': [-0.005, 0.005],
        # 'QUAD:DIAG0:285:BCTRL': [-0.005, 0.005],
        # 'QUAD:DIAG0:300:BCTRL': [-0.005, 0.005],
        # 'QUAD:DIAG0:360:BCTRL': [-0.005, 0.005],
        # 'QUAD:DIAG0:370:BCTRL': [-0.005, 0.005],
        # 'QUAD:DIAG0:390:BCTRL': [-0.005, 0.005],
    },
    constraints = IMAGE_CONSTRAINTS,
    objectives = {"total_size": "MINIMIZE"},
)
vocs.variable_names

# UCB generator

In [ ]:
# remember to set use low noise prior to false!!!
model_constructor = StandardModelConstructor(use_low_noise_prior=False)
generator = UpperConfidenceBoundGenerator(
    vocs=vocs,
    model_constructor=model_constructor,
)
generator.numerical_optimizer.max_iter = 200
# generator.max_travel_distances = [0.1] * len(vocs.variable_names)
evaluator = Evaluator(function=eval_beamsize)
X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
X.options.dump_file = run_dir + "beamsize_minimization.yml"
X

# Run Bayesian optimization to minimize beamsize

In [ ]:
for i in range(10):
    print(i)
    X.step()

# Quad Scan Emittance

In [ ]:
magnet_collection = create_magnet(area="DIAG0")
magnet_length = 1.0 # placeholder value
energy = 3.0e9 # placeholder values
scan_values = [-6.0, -3.0, -0.0] # placeholder values

quad_scan = QuadScanEmittance(
    beamline = 'SC_DIAG0',
    energy = energy, 
    magnet_collection = magnet_collection
    magnet_name = "QDG009"
    magnet_length = magnet_length
    scan_values = scan_values, 
    device_measurement = device_measurement
)

result_dict = quad_scan.measure()

In [ ]:
result